In [1]:
import lightgbm as lgb
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
dataset = pd.read_csv('golf2.txt')
dataset.head()

,Outlook,Temp.,Humidity,Wind,Decision
0,Sunny,85,85,Weak,No
1,Sunny,80,90,Strong,No
2,Overcast,83,78,Weak,Yes
3,Rain,70,96,Weak,Yes
4,Rain,68,80,Weak,Yes


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
 
features = []; categorical_features = []
num_of_columns = dataset.shape[1]
 
for i in range(0, num_of_columns):
    column_name = dataset.columns[i]
    column_type = dataset[column_name].dtypes
    
    if i != num_of_columns - 1: #skip target
        features.append(column_name)

    if column_type == 'object':
        le.fit(dataset[column_name])
        feature_classes = list(le.classes_)
        encoded_feature = le.transform(dataset[column_name])
        dataset[column_name] = pd.DataFrame(encoded_feature)

    if i != num_of_columns - 1: #skip target
        categorical_features.append(column_name)

    if i == num_of_columns - 1:
        num_of_classes = len(feature_classes)

In [4]:
y_train = dataset['Decision'].values
x_train = dataset.drop(columns=['Decision']).values

In [5]:
lgb_train = lgb.Dataset(x_train, y_train
,feature_name = features
, categorical_feature = categorical_features
)

In [6]:
params = {
'task': 'train',
'boosting_type': 'gbdt',
'objective': 'multiclass',
'num_class': num_of_classes,
'metric': 'multi_logloss',
'min_data': 1,
'verbose': -1
}

In [ ]:
gbm = lgb.train(params, lgb_train, num_boost_round=50)